# Pandas Exercises

Tamás Gál (tamas.gal@fau.de)

The latest version of this notebook is available at [https://github.com/escape2020/school2021](https://github.com/escape2020/school2021)

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as ml
import sys
plt = ml.pyplot
ml.rcParams['figure.figsize'] = (10.0, 5.0)

print(f"Python version: {sys.version}\n"
      f"Pandas version: {pd.__version__}\n"
      f"NumPy version: {np.__version__}\n"
      f"Matplotlib version: {ml.__version__}\n"
      f"seaborn version: {sns.__version__}")

Python version: 3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]
Pandas version: 2.0.1
NumPy version: 1.24.3
Matplotlib version: 3.7.1
seaborn version: 0.12.2


In [2]:
from IPython.core.magic import register_line_magic

@register_line_magic
def shorterr(line):
    """Show only the exception message if one is raised."""
    try:
        output = eval(line)
    except Exception as e:
        print("\x1b[31m\x1b[1m{e.__class__.__name__}: {e}\x1b[0m".format(e=e))
    else:
        return output
    
del shorterr

In [3]:
import warnings
warnings.filterwarnings('ignore')  # annoying UserWarnings from Jupyter/seaborn which are not fixed yet

## Exercise 1

Use the `pd.read_csv()` function to create a `DataFrame` from the dataset `data/neutrinos.csv`.

In [4]:
%shorterr neutrinos = pd.read_csv('data/neutrinos.csv')

SyntaxError: invalid syntax (<string>, line 1)


### Problems encountered

- the first few lines represent a plain header and need to be skipped
- comments are indicated with `$` at the beginning of the line
- the column separator is `:`
- the decimal delimiter is `,`
- the index column is the first one
- there is a footer to be excluded
- footer exclusion only works with the Python-engine

### Solution to exercise 1

In [5]:
!head -n 15 data/neutrinos.csv

Provided by Moritz Lotze, unformatted by Tamas Gal for teaching purposes,
Moritz would never hand out such a mess ;)

Have fun!

$ Column names
:azimuth:zenith:bjorkeny:energy:pos_x:pos_y:pos_z:proba_track:proba_cscd

$ Data
0:2,3495370211373316:1,1160038417256017:0,04899799823760986:3,3664000034332275:52,74:28,831:401,18600000000004:0,8243512974051896:0,17564870259481039
1:5,575785663044353:1,7428377336692398:0,28047099709510803:3,890000104904175:48,369:29,865:417,282:0,8183632734530938:0,18163672654690619
2:4,656124692722159:2,686909147834136:0,1198429986834526:3,2335000038146973:71,722:121,449:363,077:0,8283433133732535:0,17165668662674652
3:0,5204858401306308:1,939325715443025:0,06131500005722046:4,783999919891357:-47,592:-84,466:350,687:0,8423153692614771:0,15768463073852296
4:2,8569704309044184:1,6788966833786627:0,06146499887108803:3,983299970626831:-25,518:24,362:391,891:0,8622754491017964:0,1377245508982036
5:5,519597420755324:2,2190143556767175:0,15195700526237488:4,667799949

In [6]:
neutrinos = pd.read_csv('data/neutrinos.csv',
                        skiprows=5,
                        comment='$',
                        sep=':',
                        decimal=',',
                        index_col=0,
                        skipfooter=1,
                        engine='python')

In [7]:
neutrinos.head(3)

,azimuth,zenith,bjorkeny,energy,pos_x,pos_y,pos_z,proba_track,proba_cscd
0,2.349537,1.116004,0.048998,3.3664,52.740,28.831,401.186,0.824351,0.175649
1,5.575786,1.742838,0.280471,3.8900,48.369,29.865,417.282,0.818363,0.181637
2,4.656125,2.686909,0.119843,3.2335,71.722,121.449,363.077,0.828343,0.171657


### Check the dtypes to make sure everthing is parsed correctly (and is not an `object`-array)

In [16]:
neutrinos.dtypes  # everything's ok now ;)

azimuth        float64
zenith         float64
bjorkeny       float64
energy         float64
pos_x          float64
pos_y          float64
pos_z          float64
proba_track    float64
proba_cscd     float64
dtype: object

## Exercise 2

Create a histogram of the neutrino energies.

### Solution to exercise 2

In [ ]:
neutrinos.energy.hist(bins=100)
plt.xlabel('Neutrino energy [GeV]');
plt.ylabel('Count');

neutrinos.hist('energy', bins=100)

## Exercise 3

Use the `pd.read_csv()` function to create a `DataFrame` from the dataset `data/reco.csv`.

### Problems encountered

- need to define index column

### Solution to exercise 3

In [ ]:
reco = pd.read_csv('data/reco.csv', index_col=0)
reco.head()

## Exercise 4

Combine the `neutrinos` and `reco` `DataFrames`  into a single `DataFrame`

pd.concat()

### Problems encountered

- need to define the right axis
- identical column names should be avoided

### Solution to exercise 4

In [ ]:
data = pd.concat([neutrinos, reco.add_prefix('reco_')], axis="columns")

In [ ]:
data.head(3)

In [ ]:
data.columns

## Exercise 5

Make a scatter plot to visualise the zenith reconstruction quality.

`data = pd.concat([neutrinos reco.add_prefix('reco_')], axis="columns")`

### Problems encountered

- `DataFrame.plot()` is not suited to do scatter plots in earlier Pandas versions (inverts axis, sets weird limits etc.)

#### Sometimes it's better not to fight against `DataFrame.plot()`, just switch to Matplotlib ;)

In [ ]:
data.plot(x='zenith', y='reco_zenith', style='.');

### Solution to exercise 5, using `plt.scatter()`

In [ ]:
fig, ax = plt.subplots()
ax.scatter(data['zenith'], data['reco_zenith'], alpha=0.01);
ax.set_xlabel('True zenith');
ax.set_ylabel('Reconstructed zenith');

### Solution to exercise 5, using `plt.hist2d()`

In [ ]:
fig, ax = plt.subplots()
counts, xedges, yedges, im = ax.hist2d(data['zenith'], data['reco_zenith'], bins=50);
ax.set_xlabel('True zenith');
ax.set_ylabel('Reconstructed zenith');
fig.colorbar(im)

## Exercise 6

Create a histogram of the cascade probabilities (__`neutrinos`__ dataset: `proba_cscd` column) for the energy ranges 1-5 GeV, 5-10 GeV, 10-20 GeV and 20-100 GeV.

### Solution to exercise 6

In [ ]:
ebins = [1, 5, 10, 20, 100]
neutrinos['ebin'] = pd.cut(neutrinos.energy, ebins, labels=False)
neutrinos.hist('proba_cscd', by='ebin', bins=50);

## Exercise 7

Create a 2D histogram showing the distribution of the `x` and `y` values of the starting positions (`pos_x` and `pos_y`) of the neutrinos. This is basically a 2D plane of the starting positions.

### Solution to exercise 7

In [ ]:
fig, ax = plt.subplots()
counts, xedges, yedges, im = plt.hist2d(data.pos_x, data.pos_y, bins=100, cmap='viridis')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title('2D Plane')
fig.colorbar(im);

## Exercise 8

Check out `seaborn` (`import seaborn as sns`) and recreate the 2D histogram from Exercies 7.

### Solution to exercise 8

In [ ]:
sns.displot(data, x="pos_x", y="pos_y", cbar=True);

In [ ]:
sns.jointplot(data=data, x="pos_x", y="pos_y")

## Exercise 9

Create two histograms of the `azimuth` and `zenith` distribution side by side, in one plot (two subplots).

Try `pandas` built-in matplotlib wrapper and also the raw matplotlib library.

In [ ]:
data.head(2)

### Solution to exercise 9

In [ ]:
data.hist(['azimuth', 'zenith'], bins=100, figsize=(10, 3));

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))

ax = axes[0]
data.azimuth.hist(bins=100, ax=ax)  # zenith=0 is coming from above
ax.set_xlabel('azimuth [rad]')
ax.set_ylabel('count')

ax = axes[1]
data.zenith.hist(bins=100, ax=ax)
ax.set_xlabel('zenith [rad]')
ax.set_ylabel('count');

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))

for idx, column in enumerate(['azimuth', 'zenith']):
    data[column].hist(bins=100, ax=axes[idx])  # zenith=0 is coming from above
    axes[idx].set_xlabel(column + ' [rad]')
    axes[idx].set_ylabel('count')

## Exercise 10

Split the data into two groups: `upgoing` and `downgoing`, based on the `zenith` value (`zenith == 0` is coming from above).

Try out `sns.stripplot` to verify your "cut" on the data!

### Solution to exercise 10

Here, we are adding a new column to our dataset which contains True/False for each entry, regarding of its zenith direction

In [ ]:
data['upgoing'] = data.zenith < np.pi/2

In [ ]:
data_by_upgoing = data.groupby('upgoing')

Seaborn automatically recognises the grouped Pandas DataFrame:

In [ ]:
sns.stripplot(x="upgoing", y="zenith", data=data);

## Exercise 11

Create a combined histogram (two histograms overlayed in the same plot) for both `upgoing` and `downgoing` datasets, showing the `zenith` angle.

### Solution to exercise 11

In [ ]:
fig, ax = plt.subplots()

for upgoing, sub_data in data_by_upgoing:
    sub_data.hist('zenith', ax=ax, bins=100,
                  label='upgoing' if upgoing else 'downgoing',
                  histtype='step', linewidth=2)
ax.legend();

## Exercise: Read a KM3NeT Event File (`data/hits.h5`) and create ToT distributions of the hits for each digital optical module (dom_id) separately.

In [ ]:
!ptdump data/hits.h5

In [ ]:
hits = pd.read_hdf("data/hits.h5", "/hits")

In [ ]:
hits

In [ ]:
hits.tot.hist(bins=100);

In [ ]:
hits_by_dom_id = hits.groupby("dom_id")

In [ ]:
hits_by_dom_id

In [ ]:
hits_by_dom_id.groups.keys()

In [ ]:
for dom_id, dom_hits in hits_by_dom_id:
    print(dom_id)
    print(dom_hits)

In [ ]:
hits_by_dom_id.get_group(809548770)

In [ ]:
hits_by_dom_id["tot"].agg("mean")

In [ ]:
hits.sort_values(['dom_id', 'time']).groupby("dom_id").first()

In [ ]:
hits_by_dom_id = hits.groupby('dom_id')

fig, axes = plt.subplots(nrows=6, ncols=3, figsize=(16, 6), sharex=True, sharey=True)

hist_style = {'bins': 50, 'density': True, 'histtype': 'step', 'linewidth': 2}

for ax, (dom, _hits) in zip(axes.flatten(), hits_by_dom_id):
    ax.hist(_hits['tot'], label=f"DOM {dom}", **hist_style)
    ax.legend()
    ax.set_xlabel('tot [ns]')

### Exercise: Create a histogram of all time differences between consecutive hits calculated on each DOM independently

In [ ]:
hits.sort_values(['dom_id', 'time']).groupby('dom_id')['time'].agg('diff').hist(bins=100, log=True);

### Exercise: Examine the data for time differences below 50ns

In [ ]:
time_diffs = hits.sort_values(['dom_id', 'time']).groupby('dom_id')['time'].agg('diff')

In [ ]:
time_diffs[time_diffs < 50].hist(bins=50, log=True);

### Exercise: examine the hits on each PMT of each DOM

In [ ]:
hits_by_dom_id_and_pmt_id = hits.groupby(["dom_id", "pmt_id"])

In [ ]:
list(hits_by_dom_id_and_pmt_id.groups.keys())[:6]

In [ ]:
hits_by_dom_id_and_pmt_id.get_group((808432848, 3))

In [ ]:
hits_analysis = hits_by_dom_id_and_pmt_id["tot"].agg(["mean", "min", "max", "count"])

In [ ]:
hits_analysis

In [ ]:
hits_analysis.rename(columns={"count": "n_hits"})

### Custom aggregation functions and a word on performance

In [ ]:
rng = np.random.default_rng(42)  # initialise our random number generator

N = 1_000_000
pmt_id = rng.integers(0, 10, N)
quantum_efficiency = rng.random(N)

In [ ]:
pmt_id, quantum_efficiency

In [ ]:
df = pd.DataFrame({"pmt_id": pmt_id, "quantum_efficiency": quantum_efficiency})

In [ ]:
df.head()

In [ ]:
%timeit df.groupby(pmt_id).mean()

In [ ]:
def slow_mean(values):
    return sum(values)/len(values)

In [ ]:
%timeit df.groupby(pmt_id).agg(slow_mean)

In [ ]:
%timeit df.groupby(pmt_id).agg(slow_mean, engine="cython")

In [ ]:
%shorterr df.groupby(pmt_id).agg(slow_mean, engine="numba")

In [ ]:
def slow_mean_numba(values, index):
    return np.sum(values) / len(values)

In [ ]:
%timeit df.groupby(pmt_id).agg(slow_mean_numba, engine="numba")